In [ ]:
!pip install k3im medmnist -qq --upgrade

import sys
import os

os.environ['KERAS_BACKEND'] = 'jax'

In [4]:
import keras
import numpy as np

In [5]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [6]:
batch_size = 128
epochs = 2
def train_model(model):
    model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True), optimizer="adam", metrics=["accuracy"])
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    score = model.evaluate(x_test, y_test, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

In [7]:
from k3im.vit import ViT
model = ViT(
    image_size=(28, 28),
    patch_size=(7, 7),
    num_classes=num_classes,
    dim=32,
    depth=2,
    heads=8,
    mlp_dim=65,
    channels=1,
    dim_head=32,
    pool="mean",
)

In [8]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 28, 28, 1) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ extract_patches     │ (None, 4, 4, 49)  │       0 │ input_layer[0][0]    │
│ (ExtractPatches)    │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape (Reshape)   │ (None, 16, 49)    │       0 │ extract_patches[0][… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalization │ (None, 16, 49)    │      98 │ reshape[0][0]        │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense (Dense)       │ (None, 16, 32)    │   1,600 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 32)    │      64 │ dense[0][0]          │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ class_token_positi… │ (None, 17, 32)    │     576 │ layer_normalization… │
│ (ClassTokenPositio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 17, 32)    │  33,568 │ class_token_positio… │
│ (MultiHeadAttentio… │                   │         │ class_token_positio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add (Add)           │ (None, 17, 32)    │       0 │ class_token_positio… │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential          │ (None, 17, 32)    │   4,321 │ add[0][0]            │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_1 (Add)         │ (None, 17, 32)    │       0 │ add[0][0],           │
│                     │                   │         │ sequential[0][0]     │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 17, 32)    │  33,568 │ add_1[0][0],         │
│ (MultiHeadAttentio… │                   │         │ add_1[0][0]          │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_2 (Add)         │ (None, 17, 32)    │       0 │ add_1[0][0],         │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_1        │ (None, 17, 32)    │   4,321 │ add_2[0][0]          │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_3 (Add)         │ (None, 17, 32)    │       0 │ add_2[0][0],         │
│                     │                   │         │ sequential_1[0][0]   │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 17, 32)    │      64 │ add_3[0][0]          │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ max_pool            │ (None, 32)        │       0 │ layer_normalization… │
│ (GlobalAveragePool… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼────────────────────

 Total params: 78,510 (306.68 KB)

 Trainable params: 78,510 (306.68 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.5419 - loss: 1.3609 - val_accuracy: 0.9145 - val_loss: 0.2954
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9110 - loss: 0.3006 - val_accuracy: 0.9457 - val_loss: 0.1844
Test loss: 0.22440513968467712
Test accuracy: 0.9340999722480774


In [10]:
from k3im.simple_vit import SimpleViT
model = SimpleViT(
    image_size=(28, 28),
    patch_size=(7, 7),
    num_classes=num_classes,
    dim=32,
    depth=2,
    heads=8,
    mlp_dim=65,
    channels=1,
    dim_head=32,
    pool="mean",
)

In [11]:
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 28, 28, 1) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ extract_patches_1   │ (None, 4, 4, 49)  │       0 │ input_layer_3[0][0]  │
│ (ExtractPatches)    │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_1 (Reshape) │ (None, 16, 49)    │       0 │ extract_patches_1[0… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 49)    │      98 │ reshape_1[0][0]      │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_6 (Dense)     │ (None, 16, 32)    │   1,600 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 32)    │      64 │ dense_6[0][0]        │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_4 (Add)         │ (None, 16, 32)    │       0 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 16, 32)    │  33,568 │ add_4[0][0],         │
│ (MultiHeadAttentio… │                   │         │ add_4[0][0]          │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_5 (Add)         │ (None, 16, 32)    │       0 │ add_4[0][0],         │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_2        │ (None, 16, 32)    │   4,321 │ add_5[0][0]          │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_6 (Add)         │ (None, 16, 32)    │       0 │ add_5[0][0],         │
│                     │                   │         │ sequential_2[0][0]   │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 16, 32)    │  33,568 │ add_6[0][0],         │
│ (MultiHeadAttentio… │                   │         │ add_6[0][0]          │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_7 (Add)         │ (None, 16, 32)    │       0 │ add_6[0][0],         │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_3        │ (None, 16, 32)    │   4,321 │ add_7[0][0]          │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_8 (Add)         │ (None, 16, 32)    │       0 │ add_7[0][0],         │
│                     │                   │         │ sequential_3[0][0]   │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 32)    │      64 │ add_8[0][0]          │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ avg_pool            │ (None, 32)        │       0 │ layer_normalization… │
│ (GlobalAveragePool… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_11 (Dense)    │ (None, 10)        │     330 │ avg_pool[0][0]     

 Total params: 77,934 (304.43 KB)

 Trainable params: 77,934 (304.43 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.5103 - loss: 1.4526 - val_accuracy: 0.9015 - val_loss: 0.3366
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.8941 - loss: 0.3440 - val_accuracy: 0.9435 - val_loss: 0.1977
Test loss: 0.23583565652370453
Test accuracy: 0.9265999794006348


In [13]:
from k3im.cct import CCT

model = CCT(
    input_shape=input_shape,
    num_heads=8,
    projection_dim=32,
    kernel_size=3,
    stride=3,
    padding=2,
    transformer_units=[16, 32],
    stochastic_depth_rate=0.6,
    transformer_layers=2,
    num_classes=num_classes,
    positional_emb=False,
)

In [14]:
model.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 28, 28, 1) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ cct_tokenizer       │ (None, 9, 32)     │  19,008 │ input_layer_6[0][0]  │
│ (CCTTokenizer)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 9, 32)     │      64 │ cct_tokenizer[0][0]  │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 9, 32)     │  33,568 │ layer_normalization… │
│ (MultiHeadAttentio… │                   │         │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ stochastic_depth    │ (None, 9, 32)     │       0 │ multi_head_attentio… │
│ (StochasticDepth)   │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_9 (Add)         │ (None, 9, 32)     │       0 │ stochastic_depth[0]… │
│                     │                   │         │ cct_tokenizer[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 9, 32)     │      64 │ add_9[0][0]          │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_12 (Dense)    │ (None, 9, 16)     │     528 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_5 (Dropout) │ (None, 9, 16)     │       0 │ dense_12[0][0]       │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_13 (Dense)    │ (None, 9, 32)     │     544 │ dropout_5[0][0]      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_6 (Dropout) │ (None, 9, 32)     │       0 │ dense_13[0][0]       │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ stochastic_depth_1  │ (None, 9, 32)     │       0 │ dropout_6[0][0]      │
│ (StochasticDepth)   │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_10 (Add)        │ (None, 9, 32)     │       0 │ stochastic_depth_1[… │
│                     │                   │         │ add_9[0][0]          │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 9, 32)     │      64 │ add_10[0][0]         │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 9, 32)     │  33,568 │ layer_normalization… │
│ (MultiHeadAttentio… │                   │         │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ stochastic_depth_2  │ (None, 9, 32)     │       0 │ multi_head_attentio… │
│ (StochasticDepth)   │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_11 (Add)        │ (None, 9, 32)     │       0 │ stochastic_depth_2[… │
│                     │                   │         │ add_10[0][0]         │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 9, 32)     │      64 │ add_11[0][0]         │
│ (LayerNormalizatio… │                   │         │                    

 Total params: 88,971 (347.54 KB)

 Trainable params: 88,971 (347.54 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.6436 - loss: 1.0642 - val_accuracy: 0.9290 - val_loss: 0.2381
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.9302 - loss: 0.2362 - val_accuracy: 0.9552 - val_loss: 0.1508
Test loss: 0.1577761322259903
Test accuracy: 0.9517999887466431


In [16]:
from k3im.convmixer import ConvMixer # Check convmixer


model = ConvMixer(
    image_size=28, filters=64, depth=8, kernel_size=3, patch_size=2, num_classes=10, num_channels=1
)
model.summary()

Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 28, 28, 1) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ rescaling           │ (None, 28, 28, 1) │       0 │ input_layer_8[0][0]  │
│ (Rescaling)         │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 14, 14,    │     320 │ rescaling[0][0]      │
│                     │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation          │ (None, 14, 14,    │       0 │ conv2d_2[0][0]       │
│ (Activation)        │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalization │ (None, 14, 14,    │     256 │ activation[0][0]     │
│ (BatchNormalizatio… │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ depthwise_conv2d    │ (None, 14, 14,    │     640 │ batch_normalization… │
│ (DepthwiseConv2D)   │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_1        │ (None, 14, 14,    │       0 │ depthwise_conv2d[0]… │
│ (Activation)        │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 14, 14,    │     256 │ activation_1[0][0]   │
│ (BatchNormalizatio… │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_13 (Add)        │ (None, 14, 14,    │       0 │ batch_normalization… │
│                     │ 64)               │         │ batch_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 14, 14,    │   4,160 │ add_13[0][0]         │
│                     │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_2        │ (None, 14, 14,    │       0 │ conv2d_3[0][0]       │
│ (Activation)        │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 14, 14,    │     256 │ activation_2[0][0]   │
│ (BatchNormalizatio… │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ depthwise_conv2d_1  │ (None, 14, 14,    │     640 │ batch_normalization… │
│ (DepthwiseConv2D)   │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_3        │ (None, 14, 14,    │       0 │ depthwise_conv2d_1[… │
│ (Activation)        │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 14, 14,    │     256 │ activation_3[0][0]   │
│ (BatchNormalizatio… │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_14 (Add)        │ (None, 14, 14,    │       0 │ batch_normalization… │
│                     │ 64)               │         │ batch_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 14, 14,    │   4,160 │ add_14[0][0]       

 Total params: 43,722 (170.79 KB)

 Trainable params: 41,546 (162.29 KB)

 Non-trainable params: 2,176 (8.50 KB)

In [17]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.6886 - loss: 1.1870 - val_accuracy: 0.1045 - val_loss: 174.8289
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.9161 - loss: 0.3115 - val_accuracy: 0.1050 - val_loss: 90.8182
Test loss: 90.73978424072266
Test accuracy: 0.11349999904632568


In [18]:
from k3im.eanet import EANet
model = EANet(
    input_shape=input_shape,
    patch_size=7,
    embedding_dim=64,
    num_transformer_blocks=2,
    mlp_dim=32,
    num_heads=16,
    dim_coefficient=2,
    attention_dropout=0.5,
    projection_dropout=0.5,
    num_classes=10,
)

In [19]:
model.summary()

Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 28, 28, 1) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ patch_extract       │ (None, 16, 49)    │       0 │ input_layer_9[0][0]  │
│ (PatchExtract)      │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ patch_embedding     │ (None, 16, 64)    │   4,224 │ patch_extract[0][0]  │
│ (PatchEmbedding)    │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 64)    │     128 │ patch_embedding[0][… │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_20 (Dense)    │ (None, 16, 128)   │   8,320 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_2 (Reshape) │ (None, 16, 32, 4) │       0 │ dense_20[0][0]       │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transpose           │ (None, 32, 16, 4) │       0 │ reshape_2[0][0]      │
│ (Transpose)         │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_21 (Dense)    │ (None, 32, 16,    │     160 │ transpose[0][0]      │
│                     │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ softmax_6 (Softmax) │ (None, 32, 16,    │       0 │ dense_21[0][0]       │
│                     │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ lambda (Lambda)     │ (None, 32, 16,    │       0 │ softmax_6[0][0]      │
│                     │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_10          │ (None, 32, 16,    │       0 │ lambda[0][0]         │
│ (Dropout)           │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_22 (Dense)    │ (None, 32, 16, 4) │     132 │ dropout_10[0][0]     │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transpose_1         │ (None, 16, 32, 4) │       0 │ dense_22[0][0]       │
│ (Transpose)         │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_3 (Reshape) │ (None, 16, 128)   │       0 │ transpose_1[0][0]    │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_23 (Dense)    │ (None, 16, 64)    │   8,256 │ reshape_3[0][0]      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_11          │ (None, 16, 64)    │       0 │ dense_23[0][0]       │
│ (Dropout)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_21 (Add)        │ (None, 16, 64)    │       0 │ dropout_11[0][0],    │
│                     │                   │         │ patch_embedding[0][… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 64)    │     128 │ add_21[0][0]         │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼────────────────────

 Total params: 47,506 (185.57 KB)

 Trainable params: 47,506 (185.57 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.4249 - loss: 1.6590 - val_accuracy: 0.8365 - val_loss: 0.5266
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.7781 - loss: 0.6857 - val_accuracy: 0.9113 - val_loss: 0.3001
Test loss: 0.350763738155365
Test accuracy: 0.891700029373169


In [21]:
from k3im.gmlp import gMLPModel
model = gMLPModel(
    image_size=28,
    patch_size=7,
    embedding_dim=32,
    num_blocks=4,
    dropout_rate=0.5,
    num_classes=num_classes,
    positional_encoding=False,
    num_channels=1,
)

In [22]:
model.summary()

Model: "functional_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer_10 (InputLayer)     │ (None, 28, 28, 1)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ patches (Patches)               │ (None, 16, 49)            │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_33 (Dense)                │ (None, 16, 32)            │      1,600 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ g_mlp_layer (gMLPLayer)         │ (None, 16, 32)            │      3,568 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ g_mlp_layer_1 (gMLPLayer)       │ (None, 16, 32)            │      3,568 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ g_mlp_layer_2 (gMLPLayer)       │ (None, 16, 32)            │      3,568 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ g_mlp_layer_3 (gMLPLayer)       │ (None, 16, 32)            │      3,568 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ global_average_pooling1d_1      │ (None, 32)                │          0 │
│ (GlobalAveragePooling1D)        │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_22 (Dropout)            │ (None, 32)                │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_46 (Dense)                │ (None, 10)                │        330 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 16,202 (63.29 KB)

 Trainable params: 16,202 (63.29 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.2462 - loss: 2.0940 - val_accuracy: 0.7595 - val_loss: 0.7761
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6231 - loss: 1.1040 - val_accuracy: 0.9082 - val_loss: 0.3132
Test loss: 0.375752717256546
Test accuracy: 0.8859999775886536


In [24]:
from k3im.mlp_mixer import MixerModel
model = MixerModel(
    image_size=28,
    patch_size=7,
    embedding_dim=32,
    num_blocks=4,
    dropout_rate=0.5,
    num_classes=num_classes,
    positional_encoding=True,
    num_channels=1,
)

model.summary()

Model: "functional_30"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_15      │ (None, 28, 28, 1) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ patches_1 (Patches) │ (None, 16, 49)    │       0 │ input_layer_15[0][0] │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_47 (Dense)    │ (None, 16, 32)    │   1,600 │ patches_1[0][0]      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ position_embedding  │ (None, 16, 32)    │     512 │ dense_47[0][0]       │
│ (PositionEmbedding) │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_25 (Add)        │ (None, 16, 32)    │       0 │ dense_47[0][0],      │
│                     │                   │         │ position_embedding[… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ mlp_mixer_layer     │ (None, 16, 32)    │   1,680 │ add_25[0][0]         │
│ (MLPMixerLayer)     │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ mlp_mixer_layer_1   │ (None, 16, 32)    │   1,680 │ mlp_mixer_layer[0][… │
│ (MLPMixerLayer)     │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ mlp_mixer_layer_2   │ (None, 16, 32)    │   1,680 │ mlp_mixer_layer_1[0… │
│ (MLPMixerLayer)     │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ mlp_mixer_layer_3   │ (None, 16, 32)    │   1,680 │ mlp_mixer_layer_2[0… │
│ (MLPMixerLayer)     │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ global_average_poo… │ (None, 32)        │       0 │ mlp_mixer_layer_3[0… │
│ (GlobalAveragePool… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_31          │ (None, 32)        │       0 │ global_average_pool… │
│ (Dropout)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_64 (Dense)    │ (None, 10)        │     330 │ dropout_31[0][0]     │
└─────────────────────┴───────────────────┴─────────┴──────────────────────┘

 Total params: 9,162 (35.79 KB)

 Trainable params: 9,162 (35.79 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.2454 - loss: 2.1532 - val_accuracy: 0.7982 - val_loss: 0.6211
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.7099 - loss: 0.8873 - val_accuracy: 0.8825 - val_loss: 0.3738
Test loss: 0.42113006114959717
Test accuracy: 0.8712000250816345


In [26]:
from k3im.simple_vit_with_fft import SimpleViTFFT
model = SimpleViTFFT(image_size=28, patch_size=7, freq_patch_size=7, num_classes=num_classes, dim=32, depth=2, 
                     heads=8, mlp_dim=64, channels=1, 
                     dim_head = 16)
model.summary()

Model: "functional_34"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_24      │ (None, 28, 28, 1) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ lambda_2 (Lambda)   │ (None, 28, 28, 2) │       0 │ input_layer_24[0][0] │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ extract_patches_2   │ (None, 4, 4, 49)  │       0 │ input_layer_24[0][0] │
│ (ExtractPatches)    │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ extract_patches_3   │ (None, 4, 4, 98)  │       0 │ lambda_2[0][0]       │
│ (ExtractPatches)    │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_6 (Reshape) │ (None, 16, 49)    │       0 │ extract_patches_2[0… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_7 (Reshape) │ (None, 16, 98)    │       0 │ extract_patches_3[0… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 49)    │      98 │ reshape_6[0][0]      │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 98)    │     196 │ reshape_7[0][0]      │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_65 (Dense)    │ (None, 16, 32)    │   1,600 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_66 (Dense)    │ (None, 16, 32)    │   3,168 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 32)    │      64 │ dense_65[0][0]       │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 32)    │      64 │ dense_66[0][0]       │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_26 (Add)        │ (None, 16, 32)    │       0 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_27 (Add)        │ (None, 16, 32)    │       0 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ concatenate         │ (None, 32, 32)    │       0 │ add_26[0][0],        │
│ (Concatenate)       │                   │         │ add_27[0][0]         │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 32, 32)    │  16,800 │ concatenate[0][0],   │
│ (MultiHeadAttentio… │                   │         │ concatenate[0][0]    │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_28 (Add)        │ (None, 32, 32)    │       0 │ concatenate[0][0],   │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_17       │ (None, 32, 32)    │   4,256 │ add_28[0][0]         │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_29 (Add)        │ (None, 32, 32)    │       0 │ add_28[0][0],        │
│                     │                   │         │ sequential_17[0][0]

 Total params: 47,696 (186.31 KB)

 Trainable params: 47,696 (186.31 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.5062 - loss: 1.4556 - val_accuracy: 0.8795 - val_loss: 0.4012
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.8776 - loss: 0.3968 - val_accuracy: 0.9382 - val_loss: 0.2112
Test loss: 0.24840669333934784
Test accuracy: 0.9253000020980835


In [28]:
from k3im.simple_vit_with_register_tokens import SimpleViT_RT
model = SimpleViT_RT(image_size=28,
    patch_size=7,
    num_classes=num_classes,
    dim=32,
    depth=2,
    heads=4,
    mlp_dim=64,
    num_register_tokens=4,
    channels=1,
    dim_head=64,)

In [29]:
model.summary()

Model: "functional_38"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_27      │ (None, 28, 28, 1) │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ extract_patches_4   │ (None, 4, 4, 49)  │       0 │ input_layer_27[0][0] │
│ (ExtractPatches)    │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ reshape_8 (Reshape) │ (None, 16, 49)    │       0 │ extract_patches_4[0… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 49)    │      98 │ reshape_8[0][0]      │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_72 (Dense)    │ (None, 16, 32)    │   1,600 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 16, 32)    │      64 │ dense_72[0][0]       │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_32 (Add)        │ (None, 16, 32)    │       0 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ register_tokens     │ (None, 20, 32)    │     128 │ add_32[0][0]         │
│ (RegisterTokens)    │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 20, 32)    │  33,568 │ register_tokens[0][… │
│ (MultiHeadAttentio… │                   │         │ register_tokens[0][… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_33 (Add)        │ (None, 20, 32)    │       0 │ register_tokens[0][… │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_19       │ (None, 20, 32)    │   4,256 │ add_33[0][0]         │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_34 (Add)        │ (None, 20, 32)    │       0 │ add_33[0][0],        │
│                     │                   │         │ sequential_19[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 20, 32)    │  33,568 │ add_34[0][0],        │
│ (MultiHeadAttentio… │                   │         │ add_34[0][0]         │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_35 (Add)        │ (None, 20, 32)    │       0 │ add_34[0][0],        │
│                     │                   │         │ multi_head_attentio… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ sequential_20       │ (None, 20, 32)    │   4,256 │ add_35[0][0]         │
│ (Sequential)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_36 (Add)        │ (None, 20, 32)    │       0 │ add_35[0][0],        │
│                     │                   │         │ sequential_20[0][0]  │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 20, 32)    │      64 │ add_36[0][0]         │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ split (Split)       │ [(None, 16, 32),  │       0 │ layer_normalization

 Total params: 77,932 (304.42 KB)

 Trainable params: 77,932 (304.42 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
train_model(model)

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.5092 - loss: 1.4363 - val_accuracy: 0.8662 - val_loss: 0.4330
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8685 - loss: 0.4252 - val_accuracy: 0.9273 - val_loss: 0.2530
Test loss: 0.2882605493068695
Test accuracy: 0.9120000004768372


In [31]:
from k3im.swint import SwinTModel ########## PROBLEM

In [32]:
model = SwinTModel(
    img_size=28,
    patch_size=7,
    embed_dim=32,
    num_heads=4,
    window_size=4,
    num_mlp=4,
    qkv_bias=True,
    dropout_rate=0.2,
    shift_size=2,
    num_classes=num_classes,
    in_channels=1,
)

In [33]:
model.summary()

Model: "functional_42"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer_30 (InputLayer)     │ (None, 28, 28, 1)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ extract_patches_5               │ (None, 4, 4, 49)          │          0 │
│ (ExtractPatches)                │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ reshape_9 (Reshape)             │ (None, 16, 49)            │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ patch_embedding_1               │ (None, 16, 32)            │      2,112 │
│ (PatchEmbedding)                │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ swin_transformer                │ (None, 16, 32)            │      5,096 │
│ (SwinTransformer)               │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ swin_transformer_1              │ (None, 16, 32)            │      5,352 │
│ (SwinTransformer)               │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ patch_merging (PatchMerging)    │ (None, 4, 64)             │      8,192 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ global_average_pooling1d_3      │ (None, 64)                │          0 │
│ (GlobalAveragePooling1D)        │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_88 (Dense)                │ (None, 10)                │        650 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 21,402 (84.60 KB)

 Trainable params: 20,634 (80.60 KB)

 Non-trainable params: 768 (4.00 KB)

In [ ]:
train_model(model)